In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters) + 1 # Plus EOS marker

def findFiles(path): return glob.glob(path)

# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

# Build the category_lines dictionary, a list of lines per category
category_lines = {}
all_categories = []
for filename in findFiles('data/names/*.txt'):
    category = filename.split('/')[-1].split('.')[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

if n_categories == 0:
    raise RuntimeError('Data not found. Make sure that you downloaded data '
        'from https://download.pytorch.org/tutorial/data.zip and extract it to '
        'the current directory.')

print('# categories:', n_categories, all_categories)
print(unicodeToAscii("O'Néàl"))

# categories: 18 ['Italian', 'Korean', 'French', 'Chinese', 'Portuguese', 'Japanese', 'Vietnamese', 'Polish', 'Russian', 'Dutch', 'Scottish', 'Spanish', 'Arabic', 'Irish', 'Czech', 'English', 'German', 'Greek']
O'Neal


In [137]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.input_size = input_size
        self.output_size = output_size
        
        self.i2h = nn.Linear(n_categories+ input_size+hidden_size, hidden_size)
        self.i2o = nn.Linear(n_categories + input_size+hidden_size, output_size)
        self.o2o = nn.Linear(output_size+hidden_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)
    
    
    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input,hidden),1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden,output),1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1,self.hidden_size)

In [138]:
import torch
import torch.nn as nn

class RNN1(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [139]:
import random

# Random item from a list
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# Get a random category and random line from that category
def randomTrainingPair():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    return category, line

In [140]:
# One-hot vector for category
def categoryTensor(category):
    li = all_categories.index(category)
    tensor = torch.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor

# One-hot matrix of first to last letters (not including EOS) for input
def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

# LongTensor of second letter to end (EOS) for target
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1) # EOS
    #letter_indexes.append(58) # EOS
    return torch.LongTensor(letter_indexes)

In [141]:
# Make category, input, and target tensors from a random category, line pair
def randomTrainingExample():
    category, line = randomTrainingPair()
    category_tensor = categoryTensor(category)
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    return category_tensor, input_line_tensor, target_line_tensor

In [142]:
criterion = nn.NLLLoss()

learning_rate = 0.0005

def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = 0

    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l

    loss.backward()

    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item() / input_line_tensor.size(0)

In [143]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [144]:
rnn = RNN(n_letters, 128, n_letters)

n_iters = 10000
print_every = 50
plot_every = 50
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    total_loss += loss

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

0m 0s (50 0%) 4.1016
0m 0s (100 1%) 4.0196
0m 0s (150 1%) 4.0047
0m 1s (200 2%) 4.0096
0m 1s (250 2%) 3.9921
0m 1s (300 3%) 4.0131
0m 2s (350 3%) 3.9971
0m 2s (400 4%) 3.8185
0m 3s (450 4%) 3.9263
0m 3s (500 5%) 3.9251
0m 3s (550 5%) 3.7644
0m 4s (600 6%) 3.9461
0m 4s (650 6%) 3.8929
0m 5s (700 7%) 3.6742
0m 5s (750 7%) 3.7501
0m 5s (800 8%) 3.5491
0m 6s (850 8%) 3.6553
0m 6s (900 9%) 3.4946
0m 7s (950 9%) 3.6568
0m 7s (1000 10%) 3.6048
0m 7s (1050 10%) 3.2622
0m 7s (1100 11%) 3.8320
0m 8s (1150 11%) 3.2619
0m 8s (1200 12%) 3.0409
0m 8s (1250 12%) 3.0355
0m 9s (1300 13%) 3.6841
0m 9s (1350 13%) 3.0652
0m 9s (1400 14%) 3.0153
0m 10s (1450 14%) 3.4162
0m 10s (1500 15%) 3.4344
0m 10s (1550 15%) 2.8746
0m 10s (1600 16%) 3.1846
0m 10s (1650 16%) 3.2469
0m 11s (1700 17%) 2.9918
0m 11s (1750 17%) 3.0395
0m 12s (1800 18%) 2.8385
0m 12s (1850 18%) 2.8676
0m 12s (1900 19%) 3.1874
0m 12s (1950 19%) 3.2406
0m 13s (2000 20%) 3.0337
0m 13s (2050 20%) 3.0904
0m 13s (2100 21%) 3.2262
0m 14s (2150 21%)

In [130]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)
plt.show()

In [146]:
max_length = 20

# Sample from a category and starting letter
def sample(category, start_letter='A'):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letter)
        hidden = rnn.initHidden()

        output_name = start_letter

        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = inputTensor(letter)

        return output_name

# Get multiple samples from one category and multiple starting letters
def samples(category, start_letters='ABC'):
    for start_letter in start_letters:
        print(sample(category, start_letter))

samples('Russian', 'OLE')

samples('German', 'GER')

samples('Spanish', 'SPA')

samples('Chinese', 'CHI')

Oanee
Lane
Eona
Gare
Eann
Rare
Sarer
Parer
Aaner
Can
Han
Iarer
